# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re

import joblib
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords, wordnet
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sqlalchemy import create_engine

nltk.download(['punkt', 'wordnet', 'stopwords', 'words'], quiet=True)

True

In [2]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)

X = df.message.values
Y = df.drop(['message', 'original', 'genre'], axis=1)
labels = Y.columns
Y = Y.values

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):

    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    # Tokenize text
    tokens = nltk.word_tokenize(text)
    # Remove stop words
    tokens = [w for w in tokens if w not in stopwords.words('english')]

    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


def display_results(y_pred, y_test):
    # display results
    cr = {}
    model_avg_f1 = np.empty(len(labels))
    for i, label in enumerate(labels):
        cr[label] = classification_report(
            y_test[:, i], y_pred[:, i], labels=df[label].unique(), zero_division=0)
        score = f1_score(y_test[:, i], y_pred[:, i], labels=df[label],
                         average='weighted', zero_division=0)
        model_avg_f1[i] = score

    model_avg_f1 = np.mean(model_avg_f1)
    print(f'The model weighted f1 score is {model_avg_f1}')
    return cr

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
def create_pipeline(classifier, nr_jobs=1):
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
        ('clf', MultiOutputClassifier(classifier, n_jobs=nr_jobs))
    ])
    return pipeline

In [5]:
pipeline = create_pipeline(KNeighborsClassifier(n_jobs=-1), -1)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x000001A7E1400048>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier(algorithm='auto',
                      

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# predict on test data
y_pred = pipeline.predict(X_test)
print('Prediction done.')

pipeline_cr = display_results(y_pred, y_test)
pipeline_cr.keys()

Prediction done.
The model weighted f1 score is 0.9455942857483


dict_keys(['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report'])

In [8]:
print(pipeline_cr['medical_help'])

              precision    recall  f1-score   support

           0       0.92      1.00      0.96      6058
           1       0.00      0.00      0.00       496

    accuracy                           0.92      6554
   macro avg       0.46      0.50      0.48      6554
weighted avg       0.85      0.92      0.89      6554



### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
parameters = {
    'clf__estimator__leaf_size': [1, 5],
    'clf__estimator__n_neighbors': [6, 10, 15]
}

cv = GridSearchCV(create_pipeline(KNeighborsClassifier()),
                  param_grid=parameters, cv=2, verbose=5, n_jobs=-1)

In [10]:
cv.fit(X_train, y_train)

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  12 | elapsed:  9.8min remaining:  4.9min
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 14.7min finished


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('tfidf',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        n

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
# predict on test data
y_pred = cv.predict(X_test)
    
cv_cr = display_results(y_pred, y_test)

The model weighted f1 score is 0.9447428364364079


In [12]:
cv.cv_results_

{'mean_fit_time': array([84.57689905, 84.22890913, 79.42168999, 79.51769006, 79.30017436,
        79.46167743]),
 'std_fit_time': array([0.6609807 , 0.33998978, 0.31849122, 0.09149706, 0.46047866,
        0.17299473]),
 'mean_score_time': array([205.03830397, 205.04630351, 218.62999237, 217.54953492,
        214.18990564, 215.57705843]),
 'std_score_time': array([0.6954788 , 0.85447383, 0.66499078, 1.07846797, 0.29499078,
        0.01051652]),
 'param_clf__estimator__leaf_size': masked_array(data=[1, 1, 1, 5, 5, 5],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_clf__estimator__n_neighbors': masked_array(data=[6, 10, 15, 6, 10, 15],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'clf__estimator__leaf_size': 1, 'clf__estimator__n_neighbors': 6},
  {'clf__estimator__leaf_size': 1, 'clf__estimator__n_neighbors': 10},
  {'clf__estimato

In [13]:
cv.best_params_

{'clf__estimator__leaf_size': 1, 'clf__estimator__n_neighbors': 6}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
model = create_pipeline(RandomForestClassifier(n_jobs=-1), -1)

model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                  

In [15]:
y_pred = model.predict(X_test)

model_cr = display_results(y_pred, y_test)

The model weighted f1 score is 0.9642289762615649


In [16]:
model.get_params()

{'memory': None,
 'steps': [('tfidf',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.float64'>, encoding='utf-8',
                   input='content', lowercase=True, max_df=1.0, max_features=None,
                   min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                   smooth_idf=True, stop_words=None, strip_accents=None,
                   sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x000001A7E1400048>,
                   use_idf=True, vocabulary=None)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
    

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [18]:
parameters = {
    'tfidf__norm': [False, True],
    'tfidf__use_idf': [False, True]
}


cv = GridSearchCV(create_pipeline(RandomForestClassifier()),
                  param_grid=parameters, cv=2, verbose=5, n_jobs=-1)

In [19]:
cv.fit(X_train, y_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
C:\ProgramData\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:  5.4min remaining:  9.0min
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed:  6.6min remaining:  4.0min
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:  6.6min finished


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('tfidf',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        n

In [20]:
y_pred = cv.predict(X_test)

model_cr = display_results(y_pred, y_test)

The model weighted f1 score is 0.9650036291435184


In [21]:
cv.best_params_

{'tfidf__norm': False, 'tfidf__use_idf': True}

In [22]:
svm_model = create_pipeline(OneVsRestClassifier(LinearSVC(), n_jobs=-1), -1)

In [23]:
svm_model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                                 tokenizer=<function tokenize at 0x000001A7E1400048>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 MultiOutputClassifier(estimator=OneVsRestClassifier(estimator=LinearSVC(C=1.0,
                              

In [24]:
y_pred = svm_model.predict(X_test)

model_cr = display_results(y_pred, y_test)

The model weighted f1 score is 0.9649739683648817


In [25]:
svm_model.get_params()

{'memory': None,
 'steps': [('tfidf',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.float64'>, encoding='utf-8',
                   input='content', lowercase=True, max_df=1.0, max_features=None,
                   min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                   smooth_idf=True, stop_words=None, strip_accents=None,
                   sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x000001A7E1400048>,
                   use_idf=True, vocabulary=None)),
  ('clf',
   MultiOutputClassifier(estimator=OneVsRestClassifier(estimator=LinearSVC(C=1.0,
                                                                           class_weight=None,
                                                                           dual=True,
                                                                           fit_intercept=True,
                     

In [28]:
parameters = {
    'tfidf__norm': ['l1', 'l2'],
    'tfidf__smooth_idf': [True, False],
    'tfidf__use_idf': [True, False]
}

cv = GridSearchCV(create_pipeline(OneVsRestClassifier(LinearSVC(), n_jobs=-1), -1),
                  param_grid=parameters, cv=2, verbose=5, n_jobs=-1)

In [29]:
cv.fit(X_train, y_train)

Fitting 2 folds for each of 8 candidates, totalling 16 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  13 out of  16 | elapsed: 10.8min remaining:  2.5min
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed: 10.9min finished


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('tfidf',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        n

In [30]:
y_pred = cv.predict(X_test)

model_cr = display_results(y_pred, y_test)

The model weighted f1 score is 0.9660008659301861


In [31]:
cv.best_params_

{'tfidf__norm': 'l2', 'tfidf__smooth_idf': True, 'tfidf__use_idf': False}

### 9. Export your model as a pickle file

In [32]:
joblib.dump(cv, 'models/model.pkl')

['models/model.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.